In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

# -----------------------------------
# 1. LOAD DATA
# -----------------------------------
df = pd.read_csv("synthetic_sensor_data.csv", parse_dates=["timestamp"])

# Ensure correct ordering
df = df.sort_values(["machine_id", "timestamp"]).reset_index(drop=True)

# -----------------------------------
# 2. CREATE TARGET: fail_within_24h
# -----------------------------------
HORIZON_HOURS = 24

def add_failure_horizon_labels(group: pd.DataFrame) -> pd.DataFrame:
    group = group.sort_values("timestamp").copy()
    
    group["failure_time"] = group["timestamp"].where(group["failed"] == 1)
    group["next_failure_time"] = group["failure_time"].bfill()
    
    time_diff = group["next_failure_time"] - group["timestamp"]
    group["time_to_next_failure_hours"] = time_diff.dt.total_seconds() / 3600
    
    group["fail_within_24h"] = (
        (group["time_to_next_failure_hours"] >= 0) &
        (group["time_to_next_failure_hours"] <= HORIZON_HOURS)
    ).astype(int)
    
    return group

df = df.groupby("machine_id", group_keys=False).apply(add_failure_horizon_labels)

print(df["failed"].value_counts())
print(df["fail_within_24h"].value_counts())

# -----------------------------------
# 3. FEATURE ENGINEERING (rolling stats)
# -----------------------------------
# We have data every 10 minutes:
FREQ_MIN = 10
STEPS_PER_HOUR = int(60 / FREQ_MIN)    # 6
WINDOW_6H = 6 * STEPS_PER_HOUR         # 36
WINDOW_24H = 24 * STEPS_PER_HOUR       # 144

sensor_cols = ["temp_c", "vibration_ms2", "pressure_psi", "load_pct", "rpm"]

df = df.sort_values(["machine_id", "timestamp"]).reset_index(drop=True)

grouped = df.groupby("machine_id", group_keys=False)

for col in sensor_cols:
    # 6-hour rolling stats
    df[f"{col}_mean_6h"] = grouped[col].rolling(window=WINDOW_6H, min_periods=WINDOW_6H).mean().reset_index(level=0, drop=True)
    df[f"{col}_std_6h"]  = grouped[col].rolling(window=WINDOW_6H, min_periods=WINDOW_6H).std().reset_index(level=0, drop=True)
    df[f"{col}_max_6h"]  = grouped[col].rolling(window=WINDOW_6H, min_periods=WINDOW_6H).max().reset_index(level=0, drop=True)
    
    # 24-hour rolling stats
    df[f"{col}_mean_24h"] = grouped[col].rolling(window=WINDOW_24H, min_periods=WINDOW_24H).mean().reset_index(level=0, drop=True)
    df[f"{col}_std_24h"]  = grouped[col].rolling(window=WINDOW_24H, min_periods=WINDOW_24H).std().reset_index(level=0, drop=True)
    
    # Trend features (delta from 24h mean)
    df[f"{col}_delta_24h"] = df[col] - df[f"{col}_mean_24h"]

# Example spike feature for temperature: count of high-temp points in last 24h
df["temp_high"] = (df["temp_c"] > 80).astype(int)
df["temp_high_count_24h"] = grouped["temp_high"].rolling(window=WINDOW_24H, min_periods=WINDOW_24H).sum().reset_index(level=0, drop=True)

# -----------------------------------
# 4. CLEAN UP & DROP EARLY ROWS WITHOUT FULL HISTORY
# -----------------------------------
# Drop rows where 24h features are NaN (initial warm-up period per machine)
feature_cols = [c for c in df.columns if any(k in c for k in ["mean_6h", "std_6h", "max_6h", "mean_24h", "std_24h", "delta_24h", "temp_high_count_24h"])]

df_model = df.dropna(subset=feature_cols + ["fail_within_24h"]).copy()

print("Rows after dropping initial warm-up period:", len(df_model))

# -----------------------------------
# 5. TRAIN-TEST SPLIT (TIME-BASED)
# -----------------------------------
# We use a stratified split so both train and test contain positives.
df_train, df_test = train_test_split(
    df_model,
    test_size=0.2,
    stratify=df_model["fail_within_24h"],
    random_state=42,
)

print("Train distribution (raw):")
print(df_train["fail_within_24h"].value_counts())
print("\nTest distribution (raw):")
print(df_test["fail_within_24h"].value_counts())

# -----------------------------------
# 5B. FIX CLASS IMBALANCE (UNDERSAMPLE MAJORITY)
# -----------------------------------
train_pos = df_train[df_train["fail_within_24h"] == 1]
train_neg = df_train[df_train["fail_within_24h"] == 0]

# e.g., keep about 5× as many negatives as positives
neg_sample_size = min(len(train_neg), len(train_pos) * 5)
train_neg_sampled = train_neg.sample(n=neg_sample_size, random_state=42)

train_balanced = pd.concat([train_pos, train_neg_sampled], axis=0)
train_balanced = train_balanced.sample(frac=1.0, random_state=42)

X_train = train_balanced[feature_cols]
y_train = train_balanced["fail_within_24h"]

X_test = df_test[feature_cols]
y_test = df_test["fail_within_24h"]

print("\nBalanced train distribution:")
print(y_train.value_counts())

# -----------------------------------
# 6. TRAIN MODEL (NO class_weight needed now)
# -----------------------------------
clf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1,
)

clf.fit(X_train, y_train)

# -----------------------------------
# 7. EVALUATE WITH CUSTOM THRESHOLD
# -----------------------------------
y_proba = clf.predict_proba(X_test)[:, 1]

THRESHOLD = 0.20
y_pred = (y_proba >= THRESHOLD).astype(int)

print("\n===================================================")
print(" Evaluation Metrics (Threshold = 0.20)")
print("===================================================\n")

# Basic metrics
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# AUC uses raw probabilities
auc = roc_auc_score(y_test, y_proba)

print(f"Accuracy:      {acc:.4f}")
print(f"Precision:     {prec:.4f}")
print(f"Recall:        {rec:.4f}")
print(f"F1 Score:      {f1:.4f}")
print(f"ROC AUC Score: {auc:.4f}\n")

print("Classification Report:")
print(classification_report(y_test, y_pred, digits=3, zero_division=0))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# -----------------------------------
# 8. SAVE MODEL FOR API/APP
# -----------------------------------
model_artifact = {
    "model": clf,
    "feature_cols": feature_cols
}
joblib.dump(model_artifact, "pd_24h_model.joblib")
print("\nSaved model to pd_24h_model.joblib")


C:\Users\ABHIJITH\AppData\Local\Temp\ipykernel_1524\1325430152.py:37: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("machine_id", group_keys=False).apply(add_failure_horizon_labels)


failed
0    432000
1        50
Name: count, dtype: int64
fail_within_24h
0    424800
1      7250
Name: count, dtype: int64
Rows after dropping initial warm-up period: 424900
Train distribution (raw):
fail_within_24h
0    334120
1      5800
Name: count, dtype: int64

Test distribution (raw):
fail_within_24h
0    83530
1     1450
Name: count, dtype: int64

Balanced train distribution:
fail_within_24h
0    29000
1     5800
Name: count, dtype: int64

Classification report with threshold=0.20:
              precision    recall  f1-score   support

           0      1.000     0.990     0.995     83530
           1      0.640     1.000     0.781      1450

    accuracy                          0.990     84980
   macro avg      0.820     0.995     0.888     84980
weighted avg      0.994     0.990     0.991     84980

Confusion matrix:
[[82716   814]
 [    0  1450]]

Saved model to pd_24h_model.joblib


In [12]:
test_with_proba = df_test.copy()
test_with_proba["proba_24h"] = y_proba
test_with_proba["pred_24h"] = y_pred

top_risky = test_with_proba.sort_values("proba_24h", ascending=False).head(5)
print(top_risky[["machine_id", "timestamp", "fail_within_24h", "proba_24h"]])

        machine_id           timestamp  fail_within_24h  proba_24h
314389          36 2024-01-24 00:10:00                1        1.0
97517           11 2024-01-18 03:00:00                1        1.0
252641          29 2024-01-15 06:00:00                1        1.0
149757          17 2024-01-20 20:40:00                1        1.0
211120          24 2024-01-26 22:40:00                1        1.0


In [13]:
import json
row = top_risky.iloc[0]
feature_payload = {col: float(row[col]) for col in feature_cols}
print(json.dumps({"features": feature_payload}, indent=2))

{
  "features": {
    "temp_c_mean_6h": 53.09034997399457,
    "temp_c_std_6h": 1.4735653006910328,
    "temp_c_max_6h": 55.84817091710335,
    "temp_c_mean_24h": 53.004187741689236,
    "temp_c_std_24h": 1.4868246891417372,
    "temp_c_delta_24h": -0.4837839860909128,
    "vibration_ms2_mean_6h": 1.4155767559021806,
    "vibration_ms2_std_6h": 0.23188955796635718,
    "vibration_ms2_max_6h": 1.796030761466754,
    "vibration_ms2_mean_24h": 1.4065725545192405,
    "vibration_ms2_std_24h": 0.19728263161063844,
    "vibration_ms2_delta_24h": -0.4187337158595793,
    "pressure_psi_mean_6h": 291.4320385316424,
    "pressure_psi_std_6h": 4.679774079921187,
    "pressure_psi_max_6h": 307.90479057055666,
    "pressure_psi_mean_24h": 291.3555304356021,
    "pressure_psi_std_24h": 5.130662363775696,
    "pressure_psi_delta_24h": 3.50646884951027,
    "load_pct_mean_6h": 64.33589257972311,
    "load_pct_std_6h": 5.248372517165115,
    "load_pct_max_6h": 77.97957539323222,
    "load_pct_mean_24h"